In [1]:
using PyCall
MeCab = pyimport("MeCab")
mecab = MeCab.Tagger("-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
normarize_word(word) =
    Unicode.normalize(word, :NFKC) |>
    word -> replace(word, r"\d+" => s"0")
normarize(words) = map(normarize_word, words)
words(str) =
    mecab.parse(str) |>
    node -> split(node, "\n") |>
    node -> map(node -> split(node, "\t"), node) |>
    node -> filter(node -> length(node) == 6, node)
surface(word) = word[begin]
yomi(word) = word[2]
lemma(word) = word[3]
raw_pos(word) = word[4]
pos(word) = split(raw_pos(word), "-")[begin]

pos (generic function with 1 method)

In [2]:
using Random, StatsBase

function to_damotaagaa(word)
    if lemma(word) == "，"
        return ","
    elseif lemma(word) == "．"
        return "."
    elseif lemma(word) == "、"
        return ","
    elseif lemma(word) == "。"
        return "."
    elseif lemma(word) == "！"
        return "!"
    elseif lemma(word) == "？"
        return "?"
    end
    seed = MersenneTwister(parse(Int128, SubString(word |> lemma |> md5 |> bytes2hex, 1, 16), base=16))
    new_word = ""
    for i in 1:length(word |> yomi) * 2
        if i % 2 == 0
            v, w = zip(
                ("a", 8.2),
                ("e", 13.),
                ("i", 7.),
                ("o", 7.5),
                ("u", 2.8),
                ("aa", 4.1),
                ("ee", 7.5),
                ("ii", 3.5),
                ("oo", 3.75),
                ("uu", 1.4),
                ("ou", 0.79),
                ("ea", 0.60),
                ("ai", 0.47),
                ("ei", 0.12),
                ("oi", 0.03),
            ) |> collect |> x -> map(t -> collect(t), x)
            new_word *= sample(seed, v, Weights(w))
        else
            c, w = zip(
                ("b", 1.5),
                ("c", 2.8),
                ("d", 4.3),
                ("f", 2.2),
                ("g", 2.),
                ("h", 6.1),
                ("j", 2),
                ("k", 0.77),
                ("l", 4),
                ("m", 2.4),
                ("n", 6.7),
                ("p", 1.9),
                ("q", 0.095),
                ("r", 6.),
                ("s", 6.3),
                ("t", 9.1),
                ("v", 0.98),
                ("w", 2.4),
                ("x", 0.15),
                ("y", 0.15),
                ("z", 0.074),
                ("st", 0.91),
                ("tr", 0.30),
            ) |> collect |> x -> map(t -> collect(t), x)
            new_word *= sample(seed, c, Weights(w))
        end
    end
    return new_word
end

to_damotaagaa (generic function with 1 method)

In [3]:
function format(words)
    text = ""
    first = true
    for word in words
        if match(r"[.!?]", word) !== nothing
            text *= word
            first = true
        elseif match(r"[,]", word) !== nothing
            text *= word
        else
            if first
                text *= " " * uppercasefirst(word)
                first = false
            else
                text *= " " * word
            end
        end
    end
    return strip(text)
end

format (generic function with 1 method)

In [4]:
translate(japanese) = format(to_damotaagaa.(words(japanese)))

translate (generic function with 1 method)

In [5]:
using MD5
text = "私は空 友達です．"
translate(text), join(yomi.(words(text)), " ")

("Ciniila ta busa naacenoonoo sihe.", "ワタシ ハ ソラ トモダチ デス ．")